In [ ]:
import torch
import math
import time
import numpy as np
from datasets import load_dataset
from torch.nn.functional import softmax
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
import gradio as gr

In [ ]:
#Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
#Load Tokenizer and Dataset
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT2 lacks pad token
#Load WikiText-2
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

Using device: cuda


In [ ]:
#Tokenize
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])
#Group texts
def group_texts(examples):
    joined = sum(examples["input_ids"], [])
    chunks = [joined[i:i+128] for i in range(0, len(joined), 128)]
    return {"input_ids": chunks, "labels": chunks}

lm_dataset = tokenized.map(group_texts, batched=True)

In [ ]:
#Load and Prepare Model
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
#Training Configuration
training_args = TrainingArguments(
    output_dir="./gpt2-wikitext2",
    overwrite_output_dir=True,
    evaluation_strategy="steps",         
    save_strategy="steps",               
    eval_steps=500,                      
    save_steps=500,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    save_total_limit=2,
    logging_steps=100,
    fp16=True,
    load_best_model_at_end=True,
    seed=42,
)

c:\ProgramData\anaconda3\envs\nlp\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
#Train Model
start_time = time.time()
trainer.train()
print(f"⏱️ Training completed in {(time.time() - start_time)/60:.2f} minutes")

  0%|          | 0/55077 [00:00<?, ?it/s]

{'loss': 3.0365, 'grad_norm': 23.97475814819336, 'learning_rate': 4.990921800388547e-05, 'epoch': 0.01}
{'loss': 3.4593, 'grad_norm': 12.86766529083252, 'learning_rate': 4.981843600777094e-05, 'epoch': 0.01}
{'loss': 3.6066, 'grad_norm': 20.357196807861328, 'learning_rate': 4.972765401165641e-05, 'epoch': 0.02}
{'loss': 3.6673, 'grad_norm': 6.112000942230225, 'learning_rate': 4.963687201554188e-05, 'epoch': 0.02}
{'loss': 3.5765, 'grad_norm': 0.0, 'learning_rate': 4.954609001942735e-05, 'epoch': 0.03}


  0%|          | 0/1880 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_runtime': 62.5911, 'eval_samples_per_second': 60.072, 'eval_steps_per_second': 30.036, 'epoch': 0.03}
{'loss': 3.5734, 'grad_norm': 21.513751983642578, 'learning_rate': 4.9455308023312816e-05, 'epoch': 0.03}
{'loss': 3.5055, 'grad_norm': 12.419784545898438, 'learning_rate': 4.9364526027198285e-05, 'epoch': 0.04}
{'loss': 3.6969, 'grad_norm': 14.696779251098633, 'learning_rate': 4.927374403108376e-05, 'epoch': 0.04}
{'loss': 3.4792, 'grad_norm': 7.001405239105225, 'learning_rate': 4.918296203496923e-05, 'epoch': 0.05}
{'loss': 3.6005, 'grad_norm': 7.0876851081848145, 'learning_rate': 4.90921800388547e-05, 'epoch': 0.05}


  0%|          | 0/1880 [00:00<?, ?it/s]

...

{'eval_loss': nan, 'eval_runtime': 65.4781, 'eval_samples_per_second': 57.424, 'eval_steps_per_second': 28.712, 'epoch': 2.97}
{'loss': 2.4174, 'grad_norm': 5.487424850463867, 'learning_rate': 4.51186520689217e-07, 'epoch': 2.97}
{'loss': 2.4739, 'grad_norm': 6.322876930236816, 'learning_rate': 3.6040452457468636e-07, 'epoch': 2.98}
{'loss': 2.6272, 'grad_norm': 3.8199894428253174, 'learning_rate': 2.696225284601558e-07, 'epoch': 2.98}
{'loss': 2.8605, 'grad_norm': 6.129429817199707, 'learning_rate': 1.788405323456252e-07, 'epoch': 2.99}
{'loss': 2.6134, 'grad_norm': 5.761902809143066, 'learning_rate': 8.805853623109465e-08, 'epoch': 3.0}


  0%|          | 0/1880 [00:00<?, ?it/s]

{'eval_loss': nan, 'eval_runtime': 65.4646, 'eval_samples_per_second': 57.436, 'eval_steps_per_second': 28.718, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'train_runtime': 17158.5599, 'train_samples_per_second': 6.42, 'train_steps_per_second': 3.21, 'train_loss': 2.880725129405569, 'epoch': 3.0}
⏱️ Training completed in 285.98 minutes


In [ ]:
# Evaluate: Perplexity
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"\nPerplexity: {perplexity:.2f}")

# Print all raw eval metrics
for key, val in eval_results.items():
    print(f"{key}: {val}")

  0%|          | 0/1880 [00:00<?, ?it/s]


📊 Perplexity: nan
eval_loss: nan
eval_runtime: 49.4473
eval_samples_per_second: 76.041
eval_steps_per_second: 38.02
epoch: 3.0


In [ ]:
#Evaluate: Top-k Accuracy
def compute_top_k_accuracy(model, tokenizer, dataset, k=5, sample_size=100):
    model.eval()
    model.to("cuda" if torch.cuda.is_available() else "cpu")
    correct = 0
    total = 0

    for i, example in enumerate(dataset.select(range(sample_size))):
        input_ids = torch.tensor(example['input_ids']).unsqueeze(0).to(model.device)
        with torch.no_grad():
            outputs = model(input_ids)
        logits = outputs.logits
        next_token_logits = logits[0, -1, :]
        probs = softmax(next_token_logits, dim=0)
        top_k = torch.topk(probs, k).indices.cpu().numpy()

        if i + 1 < len(dataset):
            target_token_id = dataset[i + 1]['input_ids'][0]
            if target_token_id in top_k:
                correct += 1
            total += 1

    top_k_acc = correct / total if total else 0
    print(f"🎯 Top-{k} Accuracy: {top_k_acc*100:.2f}% ({correct}/{total})")
    return top_k_acc

# Run Top-k Accuracy
top_k_acc = compute_top_k_accuracy(model, tokenizer, lm_dataset["validation"], k=5, sample_size=100)

# Save metrics
with open("metrics.txt", "w") as f:
    f.write(f"Perplexity: {perplexity:.2f}\n")
    f.write(f"Top-5 Accuracy: {top_k_acc*100:.2f}%\n")

🎯 Top-5 Accuracy: 1.00% (1/100)


In [ ]:
#Save Model
trainer.save_model("./gpt2-wikitext2-final")
tokenizer.save_pretrained("./gpt2-wikitext2-final")
print("\n✅ Model saved to ./gpt2-wikitext2-final")


✅ Model saved to ./gpt2-wikitext2-final


In [ ]:
#Gradio Demo Interface
def predict_next_words(prompt, max_new_tokens=20, top_k=5):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=top_k,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

interface = gr.Interface(
    fn=predict_next_words,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="e.g., 'The future of AI is'", lines=2),
        gr.Slider(1, 50, value=20, step=1, label="Max New Tokens"),
        gr.Slider(1, 50, value=5, step=1, label="Top-k Sampling")
    ],
    outputs="text",
    title="🧠 GPT-2 Next Word Predictor"
)

In [23]:
# Launch Gradio
import sys
if "--no-ui" not in sys.argv:
    interface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
